In [2]:
import tensorflow as tf
import os
import sys
from tensorflow import keras
import sklearn
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cv2 import Canny
from skimage.morphology import dilation, erosion, white_tophat, skeletonize
from skimage.morphology import disk 
from skimage import transform
from sklearn.svm import SVC

In [3]:
class_names = ['MALIGNANTMASSES', 'CALC', 'BENIGNMASSES', 'NORM']
class_names_label = {class_names:i for i, class_names in enumerate(class_names)}
print(class_names_label)

IMAGE_SIZE = (224, 224, 3)

def load_data():
    main_directory = r"C:\Users\hp\Documents\CursoML\processed\todas"
    
    output = []
    images = []
    labels = []
         
    for folder in os.listdir(main_directory):
        print(folder)
        label = list(class_names_label.values())[int(folder)-1]

        for file in os.listdir(os.path.join(main_directory, folder)):
            img_path = os.path.join(os.path.join(main_directory, folder), file)

            image = io.imread(img_path, as_gray=True, plugin='matplotlib')
            image = transform.resize(image, IMAGE_SIZE)

            images.append(image)
            labels.append(label)

    images = np.array(images, dtype='float32')
    labels = np.array(labels, dtype='int32')

    output.append(images)
    output.append(labels)
        
    return output

{'MALIGNANTMASSES': 0, 'CALC': 1, 'BENIGNMASSES': 2, 'NORM': 3}


In [4]:
total_images, total_labels = load_data() 

1
2
3
4


## Modelo de red neuronal

In [5]:
import keras
#from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import batch_normalization
from keras.layers.advanced_activations import LeakyReLU

#CREAR EL MODELO

cancer_model = tf.keras.models.Sequential() #pila de capas

cancer_model.add(tf.keras.layers.Conv2D(128, kernel_size=(5,5), activation='linear', padding='same', input_shape=(224,224,3))) #capa de convolucion
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D((2, 2),padding='same')) #agrupacion
cancer_model.add(tf.keras.layers.Conv2D(128, kernel_size=(5,5), activation = 'relu'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))
cancer_model.add(tf.keras.layers.Conv2D(128, (5, 5), activation='linear', padding='same'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
cancer_model.add(tf.keras.layers.Flatten())
cancer_model.add(tf.keras.layers.Dense(128, activation='linear'))
cancer_model.add(LeakyReLU(alpha=0.1))
cancer_model.add(tf.keras.layers.Dense(4, activation='softmax')) #capa softmax para clasificacion

cancer_model.summary()

cancer_model.compile(optimizer="adam",
                     loss=keras.losses.sparse_categorical_crossentropy,
                     metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 128)     9728      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 128)     409728    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 108, 108, 128)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 128)      0         
 2D)                                                    

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = 10
skf = StratifiedKFold(n_splits=folds)
skf.get_n_splits(total_images, total_labels)

accuracy_accumulate = 0

for train_index, test_index in skf.split(total_images, total_labels):
    x_train, y_train = total_images[train_index], total_labels[train_index]
    x_test, y_test = total_images[test_index], total_labels[test_index]
    
    cancer_train = cancer_model.fit(x_train, y_train, batch_size=128, epochs=6)
    test_eval = cancer_model.evaluate(x_test, y_test)
    
    accuracy_accumulate += test_eval[1]
    print('Test loss:', test_eval[0])
    print('Test accuracy:', test_eval[1])
    
accuracy_mean = accuracy_accumulate / folds
print("\nAccuracy mean:", accuracy_mean)

Epoch 1/6
9/9 [==============================] - 392s 41s/step - loss: 3.3738 - accuracy: 0.3102
Epoch 2/6
9/9 [==============================] - 376s 41s/step - loss: 1.1382 - accuracy: 0.4966
Epoch 3/6
9/9 [==============================] - 388s 42s/step - loss: 0.7582 - accuracy: 0.7522
Epoch 4/6
9/9 [==============================] - 387s 42s/step - loss: 0.6679 - accuracy: 0.7463
Epoch 5/6
9/9 [==============================] - 388s 43s/step - loss: 0.5421 - accuracy: 0.8341
Epoch 6/6
4/4 [==============================] - 11s 2s/step - loss: 3.4992 - accuracy: 0.4035
Test loss: 3.499154567718506
Test accuracy: 0.4035087823867798
Epoch 1/6
9/9 [==============================] - 391s 42s/step - loss: 0.9699 - accuracy: 0.6771
Epoch 2/6
9/9 [==============================] - 387s 42s/step - loss: 0.7643 - accuracy: 0.7717
Epoch 3/6
9/9 [==============================] - 381s 41s/step - loss: 0.7184 - accuracy: 0.7902
Epoch 4/6
9/9 [==============================] - 397s 44s/step - l

In [115]:
test_eval = cancer_model.evaluate(test_images, test_labels)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

10/10 [==============================] - 69s 5s/step - loss: 0.6928 - accuracy: 0.8301
Test loss: 0.6927545666694641
Test accuracy: 0.8301281929016113


In [121]:
predictions = cancer_model.predict(test_images)
pred_labels = np.argmax(predictions, axis = 1)
print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        96
           1       0.69      0.86      0.77        71
           2       1.00      0.75      0.86        83
           3       0.64      0.74      0.69        62

    accuracy                           0.83       312
   macro avg       0.83      0.82      0.82       312
weighted avg       0.86      0.83      0.84       312

